In [1]:
import pandas as pd
import os
import glob
import numpy as np

np.seterr(divide='ignore', invalid='ignore')
# setting notebook
pd.set_option('display.max_columns', None)

In [2]:
# go to data-joined folder
%cd data-joined
%run xuat_combined.ipynb 
%run po_combined.ipynb 
%run dmsp.ipynb

C:\Users\baolu\Desktop\gte\notebooks\data-joined


In [3]:
#go back the tree
%cd ..

C:\Users\baolu\Desktop\gte\notebooks


In [4]:
xuat = xuat_joined.copy()

## select used cols

In [5]:
dmsp = dmsp_full[['PRODUCT CODE', ' Shelf life\n(Day) ', 'Mã chủ hàng']]

In [6]:
po = po_joined[['Số đơn hàng\n(Order No)', 'Tình trạng đơn hàng']]

## merge dmsp

In [7]:
xuat = xuat.merge(dmsp, left_on='Mã vật tư\n(Goods Code)', right_on='PRODUCT CODE', how='left').drop('PRODUCT CODE', axis=1)

## merge po

In [8]:
xuat_po = xuat.merge(po, left_on='Số đơn đặt hàng', right_on='Số đơn hàng\n(Order No)', how='left').drop('Số đơn hàng\n(Order No)', axis=1)

## merge id san

In [9]:
# load data
san = pd.read_csv('../references/san.csv')

In [10]:
xuat_san = xuat_po.merge(san, left_on='Mã KH / NCC\n(Customer Code / Vendor Code)', right_on='Code Channel', how='left').drop('Code Channel', axis=1)

## filter

In [11]:
xuat1 = xuat_san.copy()

In [12]:
xuat_mch = xuat1[(xuat1['Mã chủ hàng_x']=='GB')|(xuat1['Mã chủ hàng_x']=='HP01')|(xuat1['Mã chủ hàng_x']=='RW01')].reset_index(drop=True)

In [13]:
xuat_hp_rw = xuat_mch[(xuat_mch['Mã chủ hàng_y']=='HP')|(xuat_mch['Mã chủ hàng_y']=='RW')].reset_index(drop=True)

In [14]:
xuat_drop_khac = xuat_hp_rw[xuat_hp_rw['Name']!='Khác'].reset_index(drop=True)

In [15]:
xuat_filter_po = xuat_drop_khac[(xuat_drop_khac['Tình trạng đơn hàng'] == 'Đã bàn giao chứng từ')|(xuat_drop_khac['Tình trạng đơn hàng'] == 'Đã giao hàng')].reset_index(drop=True)

## calculate

In [16]:
cal = xuat_filter_po.copy()

In [17]:
cal['%exp'] = ((cal['Ngày hết hạn (Expiry Date)'] - cal['Ngày phiếu xuất'])/ (cal[' Shelf life\n(Day) '])) / np.timedelta64(1, 'D')

In [18]:
cal['month'] = str('Tháng ') + (pd.DatetimeIndex(cal['Ngày phiếu xuất']).month).astype(str)

In [19]:
cal['countpo'] = 1/cal.groupby('Số đơn đặt hàng')['Số đơn đặt hàng'].transform('size')

## check

In [20]:
cal[['Name','Mã chủ hàng_y']].isnull().sum()

Name             0
Mã chủ hàng_y    0
dtype: int64

In [21]:
xuat_final = cal.copy()

## to csv

In [22]:
xuat_final.to_csv('../data/processed/xuat-onine.txt', sep='\t', encoding='utf-8-sig', index=False)